In [ ]:
import os
from run_deep_gpu import train_eval
from test_deep_gpu import test
import time
import argparse

reward_file = './deep/reward'+ str(time.time()) + '.txt'
# parser = argparse.ArgumentParser()

# parser.add_argument('--mode', type=str, default='train_eval')
# parser.add_argument('--save', type=str, default='RLFX')

# parser.add_argument('--reward_file', type=str, default=reward_file)


# parser.add_argument('--currency', type=str, default='AUDUSD')
# parser.add_argument('--min_history', type=int, default=1000)
# parser.add_argument('--timespan', type=int, default=3600)
# parser.add_argument('--lag', type=int, default=32)
# parser.add_argument('--num_of_eval', type=int, default=25)

# parser.add_argument('--init_lr', type=float, default=1e-1)
# parser.add_argument('--num_of_epoch', type=int, default=150)
# parser.add_argument('--num_of_episode', type=int, default=50)

# parser.add_argument('--week_num', type=int, default=1)

# parser.add_argment('--model_path', type=str, default='best_model_AUDUSD_week1_1559273823.5407195_dropout.pth')
# parser.add_argument('--offset', type=int, default=300)
# parser.add_argument('--num_of_test', type=int, default=50)

class Config():
    def __init__(self):
        self.mode = 'train_eval'
        self.save = 'RLFX'
        self.reward_file = reward_file
        self.currency = 'AUDUSD'
        self.min_history = 1000
        self.timespan = 3600
        self.lag = 32
        self.num_of_eval = 25
        self.init_lr = 1e-1
        self.num_of_epoch = 150
        self.num_of_episode = 25
        self.week_num= 1
        self.model_path = 'best_model_AUDUSD_week1_1559273823.5407195_dropout.pth'
        self.offset = 300
        self.num_of_test = 50
        
config = Config()

In [ ]:
HOURLY_RF_RATE = 2.68 * 10 / 365.0 / 24.0

In [ ]:
import numpy as np
import pandas as pd
import torch

from run_deep_gpu import Policy
#*********************************#
from utils_deep import draw_eval_episode
#*********************************#


def test(config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    policy = Policy()
    if torch.cuda.is_available():
        policy.load_state_dict(torch.load(config.model_path))
    else:
        policy.load_state_dict(torch.load(config.model_path, map_location='cpu'))
    policy.to(device)

    d = {'bid_price': [0], 'ask_price': [0], 'action':[0]}
    df = pd.DataFrame(data=d)
    rewards_over_time = []
    g = np.zeros((config.num_of_test, 256))
    with torch.no_grad():
        accumulative_reward_test = 0
        print_reward = []
        for j in range(config.num_of_test):
            current_reward = 0
            ask = np.zeros((1, 1))
            bid = np.zeros((1, 1))
            previous_action = torch.tensor([0.0]).to(device)
            while ask.shape[0] <= config.timespan and bid.shape[0]<=3600:
                target_bid, target_ask, feature_span = draw_eval_episode(config.week_num, config.lag, config.currency,
                                                                         config.min_history, j, config.offset)
                # target_bid, target_ask, feature_span = draw_eval_episode(config.lag, config.currency,
                #                                                          config.min_history, j, config.offset)
                bid, ask, feature_span = target_bid*1e3, target_ask*1e3, feature_span
            gradient = [0] * 256
            for t in range(config.timespan):  # Don't infinite loop while learning
                state = feature_span[t]
                save_action = policy(state.float(),0.1*previous_action).to(device)
                
                for i in range(len(state)):
                    state[i] += 0.00001
                    save_action_grad = policy(state.float(),0.1*previous_action).to(device)
                    gradient[i] += (save_action_grad - save_action)/0.00001
                    state[i] -= 0.00001
                if t == config.timespan-1:
                    save_action = 0
                action = save_action - previous_action

                price = 0
                if action > 0:
                    price = ask[t]
                elif action < 0:
                    price = bid[t]
                reward = torch.sum(torch.tensor(-1.).float() * action * price).to(device)
                accumulative_reward_test += reward
                current_reward  += reward

                d = {'bid_price': [bid[t]], 'ask_price': [ask[t]], 'action':[action.item()]}
                temp_df = pd.DataFrame(data=d)
                df = df.append(temp_df)
                previous_action = save_action
            print_reward.append(current_reward)
           
            print("episode_reward",current_reward)
            gradient = [i / 3600.0 for i in gradient]
            gradient = np.array(gradient)
            g[j] = (gradient)
        #get the standard deviation of print reward
        g_avg = np.mean(g, axis=0)
        reward_np = np.array(print_reward)
        std = np.std(reward_np)
        sharpe_ratio = (np.mean(reward_np) - HOURLY_RF_RATE) / std
        print ("Testing on {} datapoint and return is {}".format(config.num_of_test, accumulative_reward_test))
        rewards_over_time.append(accumulative_reward_test)

    # Save the csv file
    currency_pair = config.model_path[-27:-21]
    
    saved_path = 'deep/result/' + currency_pair + '_week_' + str(config.week_num) +'.csv'
    # saved_path = currency_pair + '_train.csv'
    print('Saving the csv file ...')
    df.to_csv(saved_path, index = False)
    np.save('all_gradient.npy', g)
    print (sharpe_ratio)

test(config)
